# **요구사항 1: titanic dataset.py 분석하기**


In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

In [9]:
class TitanicDataset(Dataset): ## 훈련 및 검증 데이터셋용
  def __init__(self, X, y): 
    self.X = torch.FloatTensor(X) ## 입력 데이터를 FloatTensor로 변환
    self.y = torch.LongTensor(y) ## 타겟 데이터를 LongTensor로 변환

  def __len__(self):
    return len(self.X) ## 데이터셋의 크기 반환

  def __getitem__(self, idx):
    feature = self.X[idx] ## 해당 인덱스의 입력 데이터 가져오기
    target = self.y[idx] ## 해당 인덱스의 타겟 데이터 가져오기
    return {'input': feature, 'target': target} ## 'input'과 'target'으로 구성된 딕셔너리 반환

  def __str__(self):
    ## 데이터셋 정보 반환
    str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
      len(self.X), self.X.shape, self.y.shape
    )
    return str


- 이 코드는 훈련 및 검증 데이터셋용 클래스이다. 데이터 초기화, 데이터셋 크기 반환, 입력받은 인덱스의 데이터 가져오기, 데이터셋의 정보 출력을 할 수 있다.

In [10]:
class TitanicTestDataset(Dataset): ## 테스트 데이터셋용
  def __init__(self, X):
    self.X = torch.FloatTensor(X)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    feature = self.X[idx]
    return {'input': feature} ## 타겟 없이 입력 데이터만 반환

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}".format(
      len(self.X), self.X.shape
    )
    return str


- 이 코드는 테스트 데이터셋용 클래스이다. 데이터 초기화, 데이터셋 크기 반환, 입력받은 인덱스의 데이터 가져오기, 데이터셋의 정보 출력을 할 수 있다.


In [11]:
def get_preprocessed_dataset():  ## 전처리된 데이터셋 불러오기
    CURRENT_FILE_PATH = os.getcwd() 
    ##아래 코드는 jupyter notebook에서 오류가 발생하기 때문에 현재 작업 디렉토리를 이용하도록 바꿔주었다.
    ##CURRENT_FILE_PATH = os.path.dirname(os.path.abspath(__file__))

    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    train_df = pd.read_csv(train_data_path) ## 훈련 데이터 읽기
    test_df = pd.read_csv(test_data_path) ## 테스트 데이터 읽기

    all_df = pd.concat([train_df, test_df], sort=False) ## concat으로 데이터 병합

    ## 데이터 전처리 과정 실행
    all_df = get_preprocessed_dataset_1(all_df)

    all_df = get_preprocessed_dataset_2(all_df)

    all_df = get_preprocessed_dataset_3(all_df)

    all_df = get_preprocessed_dataset_4(all_df)

    all_df = get_preprocessed_dataset_5(all_df)

    all_df = get_preprocessed_dataset_6(all_df)

    ## 훈련용 입력 데이터셋
    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True) 
    ## Survived 값이 기록되어 있는 데이터만을 선택하여 훈련 데이터셋으로 사용한다. Survived 열을 제거하여 입력 데이터만 남긴다. 
    ## 모델 학습에 사용될 타겟 데이터다. 기존 인덱스를 제거하고, 새로 인덱스를 0부터 다시 부여한다.
    train_y = train_df["Survived"] ## 훈련용 타겟 데이터셋
    ## 훈련 데이터에서 타겟 값인 Survived 열을 가져온다. 

    ## 테스트용 입력 데이터셋
    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True) 
    ## Survived 값이 비어있는 데이터만 선택하여 테스트 데이터셋으로 사용한다. Survived 열을 제거하여 입력 데이터만 남긴다. 
    ## 예측을 위해 모델에 입력될 값들이다. 기존 인덱스를 제거하고, 새로 인덱스를 0부터 다시 부여한다.
    dataset = TitanicDataset(train_X.values, train_y.values) ## 훈련 데이터셋 생성
    #print(dataset)
    ## 훈련 데이터셋과 검증 데이터셋을 랜덤으로 8:2로 분할
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2]) 
    test_dataset = TitanicTestDataset(test_X.values) ## 테스트 데이터셋 생성
    #print(test_dataset)

    return train_dataset, validation_dataset, test_dataset ## 각 데이터셋 반환

- Titanic dataset을 전처리한 후, 훈련, 검증, 테스트 데이터셋을 생성하여 반환하는 작업을 수행한다. 

In [12]:
def get_preprocessed_dataset_1(all_df):
    # Pclass별 Fare 평균값을 사용하여 Fare 결측치 메우기
    ## Fare 열의 결측치를 같은 Pclass(티켓 등급) 그룹별 평균으로 채움
    
    ## Pclass별 평균 Fare 값을 계산하여 새로운 데이터프레임 생성
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    ## 컬럼 이름을 Pclass와 Fare_mean으로 변경
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    ## Pclass를 기준으로 원본 데이터와 평균 Fare 값을 결합
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    ## Fare 값이 비어있을 경우, 해당 Pclass의 평균 Fare 값으로 대체
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]

    return all_df


- 전처리 과정 1: 티켓 등급(Pclass)에 따른 Fare의 평균값을 계산하여 비어있는 Fare 값(결측값)을 해당 등급의 평균 운임 값으로 채운다.

In [13]:
def get_preprocessed_dataset_2(all_df):
    # name을 세 개의 컬럼으로 분리하여 다시 all_df에 합침
    ## Name 컬럼을 family_name, honorific, name으로 분리
    
    ## 이름을 쉼표와 점을 기준으로 세 개의 부분으로 분리
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    ## 새로 생성된 데이터프레임에 각 부분에 대한 컬럼 이름 부여
    name_df.columns = ["family_name", "honorific", "name"]
    ## 각 컬럼의 앞뒤 공백을 제거하여 깔끔한 데이터를 만듦
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    ## 기존 데이터프레임에 분리된 이름 데이터프레임을 추가
    all_df = pd.concat([all_df, name_df], axis=1)

    return all_df


- 전처리 과정 2: 승객의 이름을 세 개의 컬럼(family_name, honorific, name)으로 분리하고, 기존 데이터에 추가한다.

In [14]:
def get_preprocessed_dataset_3(all_df):
    # honorific별 Age 평균값을 사용하여 Age 결측치 메우기

    ## honorific별 중앙값을 계산하고 소수점 첫째 자리에서 반올림
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    ## 컬럼 이름을 honorific과 그에 따른 나이 평균값으로 변경
    honorific_age_mean.columns = ["honorific", "honorific_age_mean", ]
    ## honorific을 기준으로 원본 데이터와 중앙 나이 값을 결합
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    ## 나이 값이 결측값인 경우, 해당 honorific의 중앙 나이 값으로 대체
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    ## 나이 중앙값 컬럼은 더 이상 필요하지 않으므로 삭제
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    return all_df


- 전처리 과정 3: 승객의 honorific(경칭)에 따른 Age(나이)의 중앙값을 사용하여 결측된 나이 값을 채운다. 

In [15]:
def get_preprocessed_dataset_4(all_df):
    # 가족수(family_num) 컬럼 새롭게 추가
    ## 가족수 및 혼자 탑승 여부 컬럼 추가

    ## SibSp(형제자매/배우자)와 Parch(부모/자녀)를 더해 family_num(가족 수) 컬럼 생성
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]

    # 혼자탑승(alone) 컬럼 새롭게 추가
    ## 가족 수가 0이면 혼자 탑승했다는 의미로 alone 컬럼에 1을 넣음
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    ## 혼자가 아닌 경우는 0으로 처리
    ##pandas 3.0에서는 inplace=True 방식이 더 이상 제대로 동작하지 않기 때문에 이 코드로 대체하였다.
    all_df["alone"] = all_df["alone"].fillna(0) 
    ##all_df["alone"].fillna(0, inplace=True)

    # 학습에 불필요한 컬럼 제거
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    return all_df


- 전처리 과정 4: 가족 수와 혼자 탑승했는지 여부를 나타내는 컬럼을 추가하고, 학습에 불필요한 컬럼을 삭제한다.

In [24]:
def get_preprocessed_dataset_5(all_df):
    # honorific 값 개수 줄이기
    ## honorific 값 축소 및 Embarked 결측값 처리
    
    ## 자주 사용되는 honorific만 남기고 나머지는 other로 대체
    all_df.loc[
    ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
    ),
    "honorific"
    ] = "other"
    ## Embarked 값의 결측치는 missing이라는 값으로 대체
    ##pandas 3.0에서는 inplace=True 방식이 더 이상 제대로 동작하지 않기 때문에 이 코드로 대체하였다.
    all_df["Embarked"] = all_df["Embarked"].fillna("missing") 
    ##all_df["Embarked"].fillna("missing", inplace=True)

    return all_df


- 전처리 과정 5: honorific의 값 개수를 줄이고, Embarked(탑승 항구) 값의 결측치는 missing으로 처리한다.

In [17]:
def get_preprocessed_dataset_6(all_df):
    # 카테고리 변수를 LabelEncoder를 사용하여 수치값으로 변경하기
    
    ## all_df 데이터프레임에서 데이터 타입이 object(문자열)인 컬럼만 선택
    ## 이를 통해 카테고리형 변수를 식별한다.
    category_features = all_df.columns[all_df.dtypes == "object"]
    ## LabelEncoder는 각 문자열 값을 0부터 시작하는 정수값으로 매핑하여 변환한다.
    from sklearn.preprocessing import LabelEncoder
    for category_feature in category_features:
        ## 각 카테고리형 변수에 대해 LabelEncoder 생성
        le = LabelEncoder()
        ## 해당 컬럼이 문자열 타입인지 확인
        if all_df[category_feature].dtypes == "object":
          ## LabelEncoder를 사용하여 해당 컬럼의 고유한 값들을 학습(fit)
          ## 고유한 문자열 값을 각각 0부터 시작하는 정수값으로 매핑
          le = le.fit(all_df[category_feature])
          ## 학습된 LabelEncoder를 사용하여, 해당 컬럼의 문자열 값을 숫자로 변환
          ## 변환된 값은 원래의 문자열 대신 수치로 대체
          all_df[category_feature] = le.transform(all_df[category_feature])

    return all_df


- 전처리 과정 6: 카테고리 변수를 LabelEncoder를 사용하여 수치값으로 변경한다. 이 작업을 통해 모델에 입력가능한 형태로 변환된다.

In [18]:
from torch import nn
class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__() ## 부모 클래스(nn.Module)의 생성자를 호출하여 초기화

    ##neural network 정의
    self.model = nn.Sequential( ##수정 가능
      ## input, hidden1, hidden2, output
      nn.Linear(n_input, 30), ## 1-2 layer
      nn.ReLU(), ## 활성화 함수 ReLU
      nn.Linear(30, 30), ## 2-3 layer
      nn.ReLU(), ## 활성화 함수 ReLU
      nn.Linear(30, n_output), ## 3-4 layer
    )

  def forward(self, x):
    ## 순전파 함수. 입력 데이터를 받아 모델의 각 층을 통과시키는 역할
    x = self.model(x) ## 신경망을 통해 입력 x를 처리
    return x


- 2개의 hidden layer를 가지는 feedforward neural network를 설계한다. Linear 층과 ReLU 활성화 함수로 구성되어 있으며, 순전파 시 입력 데이터를 받아 최종 예측 값을 출력한다.

In [19]:
def test(test_data_loader):
  ## 테스트 함수, 테스트용 데이터 로더에서 데이터를 받아 모델 예측 수행
  print("[TEST]")
  ## 테스트 데이터 로더에서 첫 번째 배치를 가져옴
  batch = next(iter(test_data_loader))
  ## 배치 내 input 데이터의 크기(Shape)를 출력, 모델 입력으로 어떤 형태의 데이터가 들어오는지 확인 가능
  print("{0}".format(batch['input'].shape))
  ## 입력 크기 11, 출력 크기 2를 가진 MyModel 생성
  my_model = MyModel(n_input=11, n_output=2)
  ## 모델에 입력 데이터를 넣고 예측값을 계산
  ## output_batch에는 모델의 출력값이 저장됨
  output_batch = my_model(batch['input'])
  ## 예측값(output_batch)에서 가장 큰 값을 prediction_batch로 
  prediction_batch = torch.argmax(output_batch, dim=1)
  ## 예측된 값들을 892번부터 하나씩 출력
  for idx, prediction in enumerate(prediction_batch, start=892):
      print(idx, prediction.item())


- 테스트 데이터 로더로부터 데이터를 불러와 모델을 사용하여 예측을 수행한 후 각 데이터의 예측값을 출력한다. 

In [23]:
if __name__ == "__main__":
  ## 전처리된 데이터셋 불러옴
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

  ## 각 데이터셋의 크기 출력, 데이터셋이 잘 분할되었는지 확인
  print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
    len(train_dataset), len(validation_dataset), len(test_dataset)
  ))
  print("#" * 50, 1)

  ## 훈련 데이터셋에서 각 샘플의 인덱스와 입력 데이터('input'), 타겟 데이터('target') 출력
  for idx, sample in enumerate(train_dataset):
    print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

  print("#" * 50, 2)

  ## 훈련, 검증 데이터는 배치 크기 16으로 설정하고 순서를 섞음
  train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
  ## 테스트 데이터는 테스트 데이터셋 전체를 한 번에 처리하도록 설정
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

  print("[TRAIN]")
  ## 훈련 데이터의 각 배치마다 입력 데이터와 타겟 데이터의 모양을 출력
  for idx, batch in enumerate(train_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

  print("[VALIDATION]")
  ## 검증 데이터의 각 배치마다 입력 데이터와 타겟 데이터의 모양을 출력
  for idx, batch in enumerate(validation_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

  print("#" * 50, 3)

  ## test 함수에서 테스트 데이터셋을 모델에 입력하여 예측값 출력
  test(test_data_loader)

train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
################################################## 1
0 - tensor([ 3.0000,  1.0000, 25.0000,  1.0000,  0.0000,  7.7750,  2.0000, 13.3029,
         2.0000,  1.0000,  0.0000]): 0
1 - tensor([ 3.0000,  1.0000, 32.0000,  0.0000,  0.0000,  8.3625,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
2 - tensor([ 3.0000,  1.0000, 38.0000,  0.0000,  0.0000,  7.0500,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
3 - tensor([ 2.0000,  1.0000, 36.0000,  1.0000,  2.0000, 27.7500,  2.0000, 21.1792,
         2.0000,  3.0000,  0.0000]): 0
4 - tensor([ 3.0000,  1.0000, 39.0000,  0.0000,  0.0000,  7.9250,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 1
5 - tensor([ 2.0000,  1.0000, 52.0000,  0.0000,  0.0000, 13.5000,  2.0000, 21.1792,
         2.0000,  0.0000,  1.0000]): 0
6 - tensor([ 1.0000,  1.0000, 47.0000,  0.0000,  0.0000, 34.0208,  2.0000, 87.5090,
         2.0000,  0.0000,  1.0000]): 0
7 - tensor([ 3.00

- 전처리된 데이터셋을 불러오고, 훈련, 검증, 테스트 데이터셋을 생성한 후 각각의 데이터셋을 로더에 담아 학습 및 예측 과정을 수행한다.

- FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
- 위와 같은 에러가 발생했었는데, 이 에러는 pandas 라이브러리에서 DataFrame이나 Series를 복사본에 대해 값을 수정하려고 할 때 발생하는 경고라고 한다. pandas 3.0에서는 inplace = True 방식이 더 이상 제대로 동작하지 않기 때문에 코드를 수정하였다.

- NameError: name '\_\_file\_\_' is not defined
- 위와 같은 에러도 발생했었는데, Jupyter Notebook에서는 \_\_file\_\_ 변수가 정의되지 않기 때문에 현재 작업 디렉토리를 사용할 수 있도록 os.getcwd()를 사용하였다.

# **요구사항 2: titanic 딥러닝 모델 훈련 코드 및 Activation Function 변경해보기**

In [72]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from datetime import datetime
import wandb
import argparse
import os

from pathlib import Path

wandb.login()

BASE_PATH = str(Path(os.getcwd()).resolve().parent.parent)

##BASE_PATH = str(Path(__file__).resolve().parent.parent) # BASE_PATH: /Users/yhhan/git/link_dl
print(BASE_PATH, "!!!!!!!")

import sys
sys.path.append(BASE_PATH)

from _03_your_code.hw2.titanic_dataset import TitanicDataset
from _03_your_code.hw2.titanic_dataset import get_preprocessed_dataset

def get_data():
  train_X, train_y, test_X = get_preprocessed_dataset() ## 전처리된 데이터셋 가져오기



  # Create a training dataset using TitanicDataset class
  dataset = TitanicDataset(train_X.values, train_y.values)
  print(dataset)

  train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
  print(len(train_dataset), len(validation_dataset))

  train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))

  return train_data_loader, validation_data_loader


class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()

    self.model = nn.Sequential( ##활성화 함수 수정해보기
      nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
      nn.LeakyReLU(),
      nn.Dropout(0.4), ## 드롭아웃
      nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
      nn.LeakyReLU(),
      nn.Dropout(0.4), ## 드롭아웃
      nn.Linear(wandb.config.n_hidden_unit_list[1], wandb.config.n_hidden_unit_list[2]),
      nn.LeakyReLU(),
      nn.Linear(wandb.config.n_hidden_unit_list[2], n_output),
    )

  def forward(self, x):
    x = self.model(x)
    return x


def get_model_and_optimizer():
  my_model = MyModel(n_input=11, n_output=2)
  optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate) ##Gradient Descent 

  return my_model, optimizer


def training_loop(model, optimizer, train_data_loader, validation_data_loader):
  n_epochs = wandb.config.epochs
  loss_fn = nn.CrossEntropyLoss()  # Use a built-in loss function ##loss 함수 정의

  next_print_epoch = 100

  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0 ## train loss와 num_trains는 루프마다 초기화 필수
    num_trains = 0
    for train_batch in train_data_loader:
      input = train_batch['input']
      target = train_batch['target']
      #input, target = train_batch
      output_train = model(input) ## 모델 출력
      loss = loss_fn(output_train, target) 
      loss_train += loss.item()
      num_trains += 1

      optimizer.zero_grad() ## 그래디언트 초기화
      loss.backward()
      optimizer.step()

## validation loss와 num_validations 초기화 필수
    loss_validation = 0.0
    num_validations = 0
    with torch.no_grad(): ## 검증땐 그래디언트 계산 안함
      for validation_batch in validation_data_loader:
        input = validation_batch['input']
        target = validation_batch['target']
        #input, target = validation_batch
        output_validation = model(input)
        loss = loss_fn(output_validation, target)
        loss_validation += loss.item()
        num_validations += 1

    wandb.log({ ## wandb에 로그 기록
      "Epoch": epoch,
      "Training loss": loss_train / num_trains,
      "Validation loss": loss_validation / num_validations
    })

    if epoch >= next_print_epoch: ## 특정 에포크마다 출력
      print(
        f"Epoch {epoch}, "
        f"Training loss {loss_train / num_trains:.4f}, "
        f"Validation loss {loss_validation / num_validations:.4f}"
      )
      next_print_epoch += 100


def main(args):
  current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

  config = {
    'epochs': args.epochs,
    'batch_size': args.batch_size,
    'learning_rate': 1e-3,
    'n_hidden_unit_list': [30, 30, 30],
  }

  wandb.init( ## wandb 초기화
    mode="online",
    project="titanic_training",
    notes="titanic wandb experiment",
    tags=["my_model", "titanic"],
    name=current_time_str,
    config=config
  )
  print(args)
  print(wandb.config)

  train_data_loader, validation_data_loader = get_data() ## 데이터 로더 가져오기


  linear_model, optimizer = get_model_and_optimizer() ## 모델 및 옵티마이저 가져오기

  print("#" * 50, 1)

  training_loop(
    model=linear_model,
    optimizer=optimizer,
    train_data_loader=train_data_loader,
    validation_data_loader=validation_data_loader
  )
  wandb.finish() ## wandb 종료


# https://docs.wandb.ai/guides/track/config
if __name__ == "__main__":
    
  ## Jupyter 노트북에서 실행될 경우 필요한 코드
  if 'ipykernel_launcher' in sys.argv[0]:
    sys.argv = sys.argv[:1]
      
  ##argumentParser 초기화
  parser = argparse.ArgumentParser()

  parser.add_argument(
    "--wandb", action=argparse.BooleanOptionalAction, default=False, help="True or False"
  )

  parser.add_argument(
    "-b", "--batch_size", type=int, default=256, help="Batch size (int, default: 256)"
  )

  parser.add_argument(
    "-e", "--epochs", type=int, default=1_000, help="Number of training epochs (int, default:1_000)"
  )

  args = parser.parse_args()

  main(args) ## 메인 함수 실행



C:\Users\user\git\link_dl !!!!!!!


Namespace(wandb=False, batch_size=256, epochs=1000)
{'epochs': 1000, 'batch_size': 256, 'learning_rate': 0.001, 'n_hidden_unit_list': [30, 30, 30]}
Data Size: 891, Input Shape: torch.Size([891, 11]), Target Shape: torch.Size([891])
713 178
################################################## 1
Epoch 100, Training loss 0.6958, Validation loss 0.6954
Epoch 200, Training loss 0.6540, Validation loss 0.6587
Epoch 300, Training loss 0.6402, Validation loss 0.6730
Epoch 400, Training loss 0.6355, Validation loss 0.6586
Epoch 500, Training loss 0.6517, Validation loss 0.6323
Epoch 600, Training loss 0.6284, Validation loss 0.6238
Epoch 700, Training loss 0.6267, Validation loss 0.6283
Epoch 800, Training loss 0.6266, Validation loss 0.6243
Epoch 900, Training loss 0.6289, Validation loss 0.5728
Epoch 1000, Training loss 0.6080, Validation loss 0.6035


Epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇█
Training loss,██▅▆▅▃▃▃▂▂▃▃▂▃▂▂▃▂▂▂▂▁▂▁▂▁▁▂▃▂▂▁▁▂▂▂▂▁▁▁
Validation loss,▆█▇▆▅▃▄▄▃▆▃▅▃▂▂▃▄▄▂▂▃▃▄▃▁▂▄▃▂▂▂▄▃▃▂▃▂▃▂▁
Epoch,1000
Training loss,0.60799
Validation loss,0.60347


- 위 코드는 d_my_model_training_with_argparse_wandb.py 파일을 활용하여 titanic_dataset과 연결되도록 만든 코드이다. 
- get_preprocessed_dataset 함수를 통해 훈련 및 검증 데이터를 가져온다.
- MyModel 클래스에서 신경망 모델을 정의하고, 레이어와 활성화 함수를 설정한다.
- 하이퍼파라미터를 설정하고, 이를 사용하여 모델을 훈련시키고 training loss, validation loss를 계산한다.
- 위 결과물을 wandb 상에 기록하여 시각화한다.


- 활성화 함수 ELU, ReLU, PReLU, LeakyReLU를 사용하여 테스트를 해본 결과 큰 차이는 없지만 LeakyReLU가 가장 낮은 loss를 얻어냈다. 따라서 아래 문제에서도 LeakyReLU를 사용할 예정이다.

- 아래 3번 문제를 풀기 위해 최대한 loss를 줄이는 방법에 대해 많은 노력을 해봤다. layer의 개수, 뉴런 수, 활성화 함수, 하이퍼파라미터 등을 조정하는 방법도 있지만, Dropout과 BatchNorm1d과 같은 함수들도 있다는 것을 알게 됐다. 아래는 이 함수들에 대한 정보를 정리해보았다.
- BatchNorm1d: 배치정규화는 딥러닝 모델의 학습 속도를 높이고, 불안정한 학습을 안정화하기 위해 사용된다. 이를 통해 모델이 더 높은 학습률로 빠르게 수렴할 수 있으며 가중치 초기화에 덜 민감해진다. 배치 정규화는 각 배치에 대해 입력의 평균과 표준편차를 계산하여 입력데이터를 정규화하는 것이다. 특정 층의 입력값들이 학습할때 너무 크거나 작지 않도록 조정하는 역할을 한다. 이 함수를 사용하면 학습이 좀 더 안정화되고, 안정적인 학습이 가능하므로 학습 속도가 증가하고, 정규화 효과를 통해 과적합을 방지하는 데 도움이 될 수 있다.
- Dropout: 과적합을 방지하기 위한 정규화 기법이다. 학습 중에 임의로 일부 뉴런을 비활성화시켜서 모델이 특정 뉴런에 과도하게 의존하지 않도록 한다. 이를 통해 모델을 더 일반화할 수 있다. 예를 들어 Dropout(0.5)로 설정하면 학습 중에 절반의 뉴런이 임의로 비활성화된다. 따라서 특정 뉴런에 과도하게 의존하는 것을 방지하고 신경망의 가중치가 더 고르게 학습되도록 돕기 때문에 과적합을 방지할 수 있고, 뉴런을 랜덤하게 제거함으로써 모델이 더 일반화된 성능을 발휘할 수 있다.
- CrossEntropyLoss 사용 이유: 원래 이진 분류 문제는 BCEWithLogitsLoss를 사용해야 한다. 찾아보니 BCEWithLogitsLoss는 1일 확률을 구한다면, 0의 확률은 (1 - (1일 확률))인데 CrossEntropyLoss를 사용하면 0일 확률과 1일 확률을 각각 구해서 조금 더 정확도 높은 결과를 얻을 수 있다고 해서 CrossEntropyLoss를 사용하였다.

- 그리고 3번 문제의 첫번째 코드는 early stopping이 있는 코드이고, 두번째 코드는 early stopping이 없는 코드이다. 
- 이 둘을 모두 3번에 넣어놓은 이유는 아래와 같다.
- 강의 시간에 배울 때는 patience를 10번 정도로 배웠던 것 같은데, 어떤 이유에서인지 100, 200 정도로 넣어도 early stopping 되어 나온 submission 값을 kaggle에 제출하면 0.5~0.6 정도로 너무 점수가 낮게 나온다. 
- 그리고 train loss와 validation loss 결과에서 train loss는 당연히 계속 내려가지만, validation loss는 다시 상승하게 된다. 이를 막기 위해 early stopping을 걸고 submission을 얻어냈던 것인데, 과적합되어 validation loss 값이 train loss 값보다 유의미하게 높은 때의 submmission을 써야 높은 정확도가 나왔다.
- 이 점이 이해가 되지 않아 두 코드 모두 작성해두었다. 
- 두 번째 코드에서 4000 에포크정도로 학습을 진행하고 submission을 kaggle에 제출하면 0.7~0.75 사이의 점수가 나온다.
- 두 번째 코드에서 10,000 에포크로 학습을 진행하면 과적합된 결과가 나오지만, submission 결과는 0.77 정도의 점수가 나온다.

# **요구사항 3: 테스트 및 submission.csv 생성**

In [57]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from datetime import datetime
import wandb
import argparse
import os
import pandas as pd  ##submission.csv 생성을 위한 pandas
from pathlib import Path

wandb.login()

BASE_PATH = str(Path(os.getcwd()).resolve().parent.parent)

import sys
sys.path.append(BASE_PATH)

from _03_your_code.hw2.titanic_dataset import TitanicDataset
from _03_your_code.hw2.titanic_dataset import get_preprocessed_dataset
from _03_your_code.hw2.titanic_dataset import TitanicTestDataset

def get_data():
    train_X, train_y, test_X = get_preprocessed_dataset()

    # Create a training dataset using TitanicDataset class
    dataset = TitanicDataset(train_X.values, train_y.values)

    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])

    train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))

    ##test dataset에 대한 dataloader 추가
    test_dataset = TitanicTestDataset(test_X.values)
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=wandb.config.batch_size, shuffle=True)
    print(len(train_dataset), len(validation_dataset), len(test_dataset))

    return train_data_loader, validation_data_loader, test_data_loader

class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()

        self.model = nn.Sequential( ##활성화 함수 수정해보기
            nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
            nn.BatchNorm1d(wandb.config.n_hidden_unit_list[0]),
            nn.LeakyReLU(),
            nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
            nn.BatchNorm1d(wandb.config.n_hidden_unit_list[1]),
            nn.LeakyReLU(),
            nn.Dropout(0.4),
            nn.Linear(wandb.config.n_hidden_unit_list[1], wandb.config.n_hidden_unit_list[2]),
            nn.BatchNorm1d(wandb.config.n_hidden_unit_list[2]),
            nn.LeakyReLU(),
            nn.Linear(wandb.config.n_hidden_unit_list[2], n_output),
        )

    def forward(self, x):
        x = self.model(x)
        return x

def get_model_and_optimizer():
    my_model = MyModel(n_input=11, n_output=2)
    optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)

    return my_model, optimizer

def training_loop(model, optimizer, train_data_loader, validation_data_loader, test_data_loader):
    n_epochs = wandb.config.epochs
    loss_fn = nn.CrossEntropyLoss()  ##loss 함수 수정해보기
    next_print_epoch = 100
    best_validation_loss = float('inf')  ## early stop을 위한 변수
    early_stop_patience = 200  ## 몇 epoch 동안 개선이 없으면 멈출지 결정
    patience_counter = 0

    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        num_trains = 0
        for train_batch in train_data_loader:
            input = train_batch['input']
            target = train_batch['target']
            output_train = model(input)
            loss = loss_fn(output_train, target)
            loss_train += loss.item()
            num_trains += 1

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        loss_validation = 0.0
        num_validations = 0
        with torch.no_grad():
            for validation_batch in validation_data_loader:
                input = validation_batch['input']
                target = validation_batch['target']
                output_validation = model(input)
                loss = loss_fn(output_validation, target)
                loss_validation += loss.item()
                num_validations += 1

        wandb.log({
            "Epoch": epoch,
            "Training loss": loss_train / num_trains,
            "Validation loss": loss_validation / num_validations
        })

        if epoch >= next_print_epoch:
            print(
                f"Epoch {epoch}, "
                f"Training loss {loss_train / num_trains:.4f}, "
                f"Validation loss {loss_validation / num_validations:.4f}"
            )
            next_print_epoch += 100

        ## Early stopping 조건 확인
        avg_validation_loss = loss_validation / num_validations
        if avg_validation_loss < best_validation_loss:
            best_validation_loss = avg_validation_loss
            patience_counter = 0  ## 개선되면 patience 초기화
        else:
            patience_counter += 1

        if patience_counter >= early_stop_patience:
            print(f"Early stopping at epoch {epoch}")
            break

    ## 테스트 데이터에 대해 예측하고 submission.csv 생성
    with torch.no_grad():
        predictions = []
        for test_batch in test_data_loader:
            input = test_batch['input']
            output = model(input)
            predicted_classes = output.argmax(dim=1)  ## 각 클래스 확률 중 가장 큰 값 선택
            predictions.extend(predicted_classes.tolist())

    submission = pd.DataFrame({
        ## 예시로 PassengerId를 892번부터 시작하도록 설정
        'PassengerId': list(range(892, 892 + len(predictions))),  
        'Survived': predictions
    })
    submission.to_csv('submission.csv', index=False)
    print("submission.csv saved.")

def main(args):
    current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

    config = {
        'epochs': args.epochs,
        'batch_size': args.batch_size,
        'learning_rate': 1e-3,
        'n_hidden_unit_list': [128, 64, 32],
    }

    wandb.init(
        mode="online",
        project="titanic_training",
        notes="titanic wandb experiment",
        tags=["my_model", "titanic"],
        name=current_time_str,
        config=config
    )
    print(args)
    print(wandb.config)

    train_data_loader, validation_data_loader, test_data_loader = get_data()

    linear_model, optimizer = get_model_and_optimizer()

    training_loop(
        model=linear_model,
        optimizer=optimizer,
        train_data_loader=train_data_loader,
        validation_data_loader=validation_data_loader,
        test_data_loader=test_data_loader
    )
    wandb.finish()

if __name__ == "__main__":
    
    if 'ipykernel_launcher' in sys.argv[0]:
        sys.argv = sys.argv[:1]
      
    parser = argparse.ArgumentParser()

    parser.add_argument(
        "--wandb", action=argparse.BooleanOptionalAction, default=False, help="True or False"
    )

    parser.add_argument(
        "-b", "--batch_size", type=int, default=256, help="Batch size (int, default: 256)"
    )

    parser.add_argument(
        "-e", "--epochs", type=int, default=10_000, help="Number of training epochs (int, default:10_000)"
    )

    args = parser.parse_args()

    main(args)


Namespace(wandb=False, batch_size=256, epochs=10000)
{'epochs': 10000, 'batch_size': 256, 'learning_rate': 0.001, 'n_hidden_unit_list': [128, 64, 32]}
713 178 418
Epoch 100, Training loss 0.6211, Validation loss 0.6597
Epoch 200, Training loss 0.6008, Validation loss 0.6567
Epoch 300, Training loss 0.5740, Validation loss 0.6298
Epoch 400, Training loss 0.5686, Validation loss 0.6241
Epoch 500, Training loss 0.5592, Validation loss 0.6111
Epoch 600, Training loss 0.5312, Validation loss 0.5893
Epoch 700, Training loss 0.5174, Validation loss 0.5721
Epoch 800, Training loss 0.5030, Validation loss 0.6046
Epoch 900, Training loss 0.5006, Validation loss 0.5674
Epoch 1000, Training loss 0.4815, Validation loss 0.5844
Epoch 1100, Training loss 0.4881, Validation loss 0.5721
Epoch 1200, Training loss 0.4756, Validation loss 0.5799
Epoch 1300, Training loss 0.4840, Validation loss 0.5270
Epoch 1400, Training loss 0.4552, Validation loss 0.5413
Epoch 1500, Training loss 0.4677, Validation los

Epoch,▁▁▁▁▁▂▂▂▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█████
Training loss,██▇▆▅▅▆▆▆▅▅▅▄▄▄▄▃▄▄▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▁▂▁▁
Validation loss,██▇▇▅▅▅▄▅▆▄▅▄▅▄▄▅▄▄▃▄▃▃▂▂▂▂▃▃▃▃▂▂▂▃▃▁▂▃▂
Epoch,1678
Training loss,0.43351
Validation loss,0.51972


In [2]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from datetime import datetime
import wandb
import argparse
import os
import pandas as pd  ##submission.csv 생성을 위한 pandas
from pathlib import Path

wandb.login()

BASE_PATH = str(Path(os.getcwd()).resolve().parent.parent)

import sys
sys.path.append(BASE_PATH)

from _03_your_code.hw2.titanic_dataset import TitanicDataset
from _03_your_code.hw2.titanic_dataset import get_preprocessed_dataset
from _03_your_code.hw2.titanic_dataset import TitanicTestDataset

def get_data():
    train_X, train_y, test_X = get_preprocessed_dataset()

    # Create a training dataset using TitanicDataset class
    dataset = TitanicDataset(train_X.values, train_y.values)

    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])

    train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))

    ##test dataset에 대한 dataloader 추가
    test_dataset = TitanicTestDataset(test_X.values)
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))
    print(len(train_dataset), len(validation_dataset), len(test_dataset))

    return train_data_loader, validation_data_loader, test_data_loader

class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()

        self.model = nn.Sequential( ##활성화 함수 수정해보기
            nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
            nn.BatchNorm1d(wandb.config.n_hidden_unit_list[0]),
            nn.LeakyReLU(),
            nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
            nn.BatchNorm1d(wandb.config.n_hidden_unit_list[1]),
            nn.LeakyReLU(),
            nn.Dropout(0.4),
            nn.Linear(wandb.config.n_hidden_unit_list[1], wandb.config.n_hidden_unit_list[2]),
            nn.BatchNorm1d(wandb.config.n_hidden_unit_list[2]),
            nn.LeakyReLU(),
            nn.Dropout(0.4),
            nn.Linear(wandb.config.n_hidden_unit_list[2], n_output),
        )

    def forward(self, x):
        x = self.model(x)
        return x

def get_model_and_optimizer():
    ##my_model = MyModel(n_input=11, n_output=1)
    my_model = MyModel(n_input=11, n_output=2)
    optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)

    return my_model, optimizer

def training_loop(model, optimizer, train_data_loader, validation_data_loader, test_data_loader):
    n_epochs = wandb.config.epochs
    ##loss_fn = nn.BCEWithLogitsLoss()  ##loss 함수 수정해보기    
    loss_fn = nn.CrossEntropyLoss()  ##loss 함수 수정해보기
    next_print_epoch = 100

    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        num_trains = 0
        for train_batch in train_data_loader:
            input = train_batch['input']
            target = train_batch['target']
            ##target = target.unsqueeze(1)
            output_train = model(input)
            loss = loss_fn(output_train, target)

            ##loss = loss_fn(output_train, target.float())
            loss_train += loss.item()
            num_trains += 1

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        loss_validation = 0.0
        num_validations = 0
        with torch.no_grad():
            for validation_batch in validation_data_loader:
                input = validation_batch['input']
                target = validation_batch['target']
                ##target = target.unsqueeze(1)
                output_validation = model(input)
                loss = loss_fn(output_validation, target)
                ##loss = loss_fn(output_validation, target.float())
                loss_validation += loss.item()
                num_validations += 1

        wandb.log({
            "Epoch": epoch,
            "Training loss": loss_train / num_trains,
            "Validation loss": loss_validation / num_validations
        })

        if epoch >= next_print_epoch:
            print(
                f"Epoch {epoch}, "
                f"Training loss {loss_train / num_trains:.4f}, "
                f"Validation loss {loss_validation / num_validations:.4f}"
            )
            next_print_epoch += 100

    ## 테스트 데이터에 대해 예측하고 submission.csv 생성
    with torch.no_grad():
        predictions = []
        for test_batch in test_data_loader:
            input = test_batch['input']
            output = model(input)
            predicted_classes = output.argmax(dim=1)  ## 각 클래스 확률 중 가장 큰 값 선택
            predictions.extend(predicted_classes.tolist())


    submission = pd.DataFrame({
        ## 예시로 PassengerId를 892번부터 시작하도록 설정
        'PassengerId': list(range(892, 892 + len(predictions))),  
        'Survived': predictions
    })
    submission.to_csv('submission.csv', index=False)
    print("submission.csv saved.")

def main(args):
    current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

    config = {
        'epochs': args.epochs,
        'batch_size': args.batch_size,
        'learning_rate': 1e-3,
        'n_hidden_unit_list': [128, 64, 32],
    }

    wandb.init(
        mode="online",
        project="titanic_training",
        notes="titanic wandb experiment",
        tags=["my_model", "titanic"],
        name=current_time_str,
        config=config
    )
    print(args)
    print(wandb.config)

    train_data_loader, validation_data_loader, test_data_loader = get_data()

    linear_model, optimizer = get_model_and_optimizer()

    training_loop(
        model=linear_model,
        optimizer=optimizer,
        train_data_loader=train_data_loader,
        validation_data_loader=validation_data_loader,
        test_data_loader=test_data_loader
    )
    wandb.finish()

if __name__ == "__main__":
    
    if 'ipykernel_launcher' in sys.argv[0]:
        sys.argv = sys.argv[:1]
      
    parser = argparse.ArgumentParser()

    parser.add_argument(
        "--wandb", action=argparse.BooleanOptionalAction, default=False, help="True or False"
    )

    parser.add_argument(
        "-b", "--batch_size", type=int, default=256, help="Batch size (int, default: 256)"
    )

    parser.add_argument(
        "-e", "--epochs", type=int, default=4_000, help="Number of training epochs (int, default:4_000)"
    )

    args = parser.parse_args()

    main(args)


Namespace(wandb=False, batch_size=256, epochs=4000)
{'epochs': 4000, 'batch_size': 256, 'learning_rate': 0.001, 'n_hidden_unit_list': [128, 64, 32]}
713 178 418
Epoch 100, Training loss 0.6549, Validation loss 0.6339
Epoch 200, Training loss 0.6435, Validation loss 0.6182
Epoch 300, Training loss 0.6281, Validation loss 0.6026
Epoch 400, Training loss 0.6066, Validation loss 0.5985
Epoch 500, Training loss 0.6027, Validation loss 0.5734
Epoch 600, Training loss 0.5972, Validation loss 0.5591
Epoch 700, Training loss 0.6118, Validation loss 0.5813
Epoch 800, Training loss 0.6058, Validation loss 0.5777
Epoch 900, Training loss 0.5915, Validation loss 0.5678
Epoch 1000, Training loss 0.5734, Validation loss 0.5632
Epoch 1100, Training loss 0.5643, Validation loss 0.5527
Epoch 1200, Training loss 0.5755, Validation loss 0.5573
Epoch 1300, Training loss 0.5705, Validation loss 0.5394
Epoch 1400, Training loss 0.5696, Validation loss 0.5621
Epoch 1500, Training loss 0.5758, Validation loss 

Epoch,▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇███
Training loss,█▅▆▆▆▅▅▄▄▄▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▃▂▂▂▁▂▂▂▁▁
Validation loss,█▆▆▅▅▅▄▅▄▅▄▄▃▄▄▃▃▃▃▂▃▃▃▂▂▃▂▃▃▂▂▂▂▃▃▂▁▂▂▂
Epoch,4000
Training loss,0.45205
Validation loss,0.45864


- 5000, 10000번 정도의 에포크를 주고 모델을 많이 학습시켜본 결과, 경험적으로 4000 에포크 정도일 때가 제일 적합하다고 생각했다. 과적합이 크게 발생하지 않으면서도 과적합된 결과의 submission 값과 큰 차이가 나지 않았기 때문이다. 
- 과적합된 모델이 더 나은 예측 결과를 내는 이유에 대해서 찾아본 결과, 훈련 데이터에서 학습한 복잡한 패턴들이 테스트 데이터에서 운좋게 맞아 떨어지는 경우가 많이 생겨서 더 나은 결과를 낼 수 있다고 한다. 하지만 일반화 능력이 떨어지기 때문에 좋은 모델이라고 볼 수는 없으므로 위 결과처럼 train loss와 validation loss가 크게 차이나지 않는 결과가 나오는 것이 좋을 것 같다.

# **요구사항 4: submission.csv 제출 및 등수 확인**


최고 점수
![윤정민_titanic_score](https://raw.githubusercontent.com/yjmini/deepLearning24-2/main/윤정민_titanic_score.PNG)


숙제 후기: 이번 과제를 하면서 하이퍼 파라미터, 활성화 함수 등 여러 조건들을 변경해보면서 loss를 줄이고 정확도를 높이는 방법에 대해서 알 수 있었다. Dropout과 BatchNorm1d의 기능과 장점에 대해서도 알 수 있었다. 학교 수업에서 보던 것처럼 early stop을 통해 최적의 상황에서 멈추고 좋은 결과를 얻어보려고 했는데, 잘 되지 않아서 아쉽다. 처음 kaggle에 제출했을 때는 점수가 0.6정도가 나오고, loss 결과들도 0.6으로 꽤 높게 나왔었다. loss를 최대한 줄일 수 있는 방법으로 여러 방법을 시도해보고 많은 모델을 만들어본 결과, loss를 0.4까지 줄일 수 있게 돼서 뿌듯했다. training loss 뿐만 아니라 validation loss도 같이 비슷하게 줄었을 때의 기분은 정말 좋았다. kaggle 아이디를 여러 개 만들어서 많이 제출해봤는데 점수가 0.775점까지 나올 수 있었고, 과적합 없는 4000번의 에포크를 거친 모델에서도 대부분 0.75정도가 나와서 이정도면 나름의 결과를 얻어낸 것이라는 생각이 들었다. 코드를 작성할 때도 계속 에러가 생겨서 힘들었지만 다행히 잘 해결했다. 그리고 BCEWithLogits를 사용하지 않고 CrossEntropy를 사용한다는 점도 신기했다. 실제로 두 함수를 모두 사용해보았는데 CrossEntropy를 사용했을 때의 결과가 좀 더 좋았다. 이런 경험을 하고 나니 딥러닝을 할 때, 하이퍼 파라미터나 loss 함수 등 정말 많은 조건들을 최적화 하는 것이 제일 어려울 것 같다고 느꼈다. 